<center> <h1> 从安然公司邮件中发现欺诈证据-by hawli </h1> </center>
<center> <h4> Udacity Data Analyst Nanodegree P5 </h4> </center>
<h4> 数据集背景 </h4>
安然电子邮件和财务数据集是关于安然公司的相关信息，安然是能源、商品和服务公司公司，由于惯性商业欺诈，2001年12月因破产而破产。在公司崩溃之后，联邦能源监管委员会在2000-2002年公布了安然公司高管们发送和接收的160万封电子邮件。经过多次关于电子邮件敏感性的投诉，FERC纠正了大部分电子邮件，但仍有大约50万人可供公众使用。电子邮件和财务数据包含电子邮件本身，例如从每个人收到的和发送的数字，以及包括薪资和股票期权的财务信息。

安然数据集已经成为机器学习实践者的有价值的培训和测试工具，可以从数据中的特征中尝试和开发可以识别兴趣者（POI）的模型。感兴趣的人是最终在安然调查中被欺诈或犯罪活动的人，包括几名高层管理人员。该项目的目标是创建一个可以分离POI的机器学习模型。

我使用了安然公司公布的原始数据。调查安然数据集的最终目标是能够提前预测欺诈或不安全的商业行为，有关人员可以受到惩罚，无辜人员可以避免伤害。

<h2> 异常值检测和数据清洗 </h2>

In [1]:
#!/usr/bin/python
import pandas as pd
import numpy as np
%matplotlib inline
from __future__ import division
import pprint
import matplotlib.pyplot as plt
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
#from tester import dump_classifier_and_data

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

df = pd.DataFrame.from_dict(data_dict, orient='index')
df = df.replace('NaN', np.nan)

In [2]:
# 检查缺失值
data_points = len(data_dict)
poi_count = 0
non_poi_count = 0
missing_value_map = { 'bonus': {'count':0, 'poi':0}, 'deferral_payments': {'count':0, 'poi':0},
    'deferred_income': {'count':0, 'poi':0},'director_fees': {'count':0, 'poi':0}, 
    'exercised_stock_options': {'count':0, 'poi':0}, 'total_payments': {'count':0, 'poi':0},
    'expenses': {'count':0, 'poi':0}, 'loan_advances': {'count':0, 'poi':0},
    'long_term_incentive': {'count':0, 'poi':0}, 'restricted_stock_deferred': {'count':0, 'poi':0},
    'other': {'count':0, 'poi':0}, 'restricted_stock': {'count':0, 'poi':0}, 
    'total_stock_value': {'count':0, 'poi':0}, 'salary': {'count':0, 'poi':0}, 
    'email_address': {'count':0, 'poi':0}, 'from_messages': {'count':0, 'poi':0}, 
    'from_poi_to_this_person': {'count':0, 'poi':0}, 'shared_receipt_with_poi': {'count':0, 'poi':0},
    'from_this_person_to_poi': {'count':0, 'poi':0}, 'to_messages': {'count':0, 'poi':0} }


for person, features in data_dict.iteritems():    
    isPoi = False    
    if features['poi'] == True:
        poi_count += 1
        isPoi = True
    else:
        non_poi_count += 1
    for name, value in features.iteritems():         
        if value == 'NaN':
            missing_value_map[name]['count'] += 1
            if isPoi:
                missing_value_map[name]['poi'] += 1

print "数据点总数:\t", data_points 
print "POI数量:\t\t", poi_count
print "非POI数量:\t", non_poi_count
print "POI率:\t\t", poi_count/data_points
print "特征总数:\t", len(data_dict[data_dict.keys()[0]])


significant_missing_values = []
significant_poi_values = []

#print "{:<25} {:<20} {:<10}".format('Feature','missing','poi')
for feature, values in missing_value_map.iteritems():
    missing_ratio = values['count']/data_points
    if missing_ratio > 0.5:
        significant_missing_values.append(feature)
    poi_ratio = values['poi']/poi_count
    if poi_ratio > 0.5:
        significant_poi_values.append(feature)
    #print "{:<25} {:<20} {:<10}".format(feature, values['count'], values['poi'])

print "缺失值超过50%的特征:", significant_missing_values
print "POI中缺失值超过50%的特征:", significant_poi_values

数据点总数:	146
POI数量:		18
非POI数量:	128
POI率:		0.123287671233
特征总数:	21
缺失值超过50%的特征: ['deferral_payments', 'loan_advances', 'restricted_stock_deferred', 'deferred_income', 'long_term_incentive', 'director_fees']
POI中缺失值超过50%的特征: ['deferral_payments', 'loan_advances', 'restricted_stock_deferred', 'director_fees']


调查的数据总量为146个，共21个特征，POI率为0.12，由于21个特征中有多个特征缺失值太多，因此选定余下几个特征进行数据探索。

In [3]:
# Explore dataset

financial_features = ['salary', 'total_payments', 'bonus', 
                      'deferred_income', 'total_stock_value', 'expenses', 
                      'exercised_stock_options', 'other', 'restricted_stock']
#'email_address',
email_features = ['to_messages', 'from_poi_to_this_person', 
                  'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']

features_list = ['poi'] + financial_features + email_features
df = df[features_list]
print "财务特征数量:\t", len(financial_features)
print "邮件特征数量:\t", len(email_features)

财务特征数量:	9
邮件特征数量:	5


In [4]:
# 去除财务表格上的异常值，这两行均非个人数据
df.drop(axis=0, labels=['TOTAL','THE TRAVEL AGENCY IN THE PARK'], inplace=True)

In [39]:
from sklearn.preprocessing import Imputer

# 为财务特征缺失值填充为0 
df[financial_features] = df[financial_features].fillna(0)

# 为邮件特征填充均值
imp = Imputer(missing_values='NaN', strategy = 'mean', axis=0)

df_poi = df[df['poi'] == True];
df_nonpoi = df[df['poi']==False]

df_poi.loc[:, email_features] = imp.fit_transform(df_poi.loc[:,email_features]);
df_nonpoi.loc[:, email_features] = imp.fit_transform(df_nonpoi.loc[:,email_features]);
df = df_poi.append(df_nonpoi)

In [6]:
# 查找是否有其他异常数据
IQR = df.quantile(q=0.75) - df.quantile(q=0.25)
first_quartile = df.quantile(q=0.25)
third_quartile = df.quantile(q=0.75)
outliers = df[(df>(third_quartile + 1.5*IQR) ) | (df<(first_quartile - 1.5*IQR) )].count(axis=1)
outliers.sort_values(axis=0, ascending=False, inplace=True)
outliers.head(10)

LAY KENNETH L         12
FREVERT MARK A         9
BELDEN TIMOTHY N       8
SKILLING JEFFREY K     8
LAVORATO JOHN J        7
KEAN STEVEN J          7
DELAINEY DAVID W       6
WHALLEY LAWRENCE G     6
KITCHEN LOUISE         6
BAXTER JOHN C          6
dtype: int64

In [7]:
# 剔除非poi的异常值
df.drop(axis=0, labels=['FREVERT MARK A', 'LAVORATO JOHN J','KEAN STEVEN J', 
                        'WHALLEY LAWRENCE G', 'KITCHEN LOUISE','BAXTER JOHN C'], inplace=True)

In [8]:
# 查看剔除后的数据点
len(df)
df['poi'].value_counts()

False    120
True      18
Name: poi, dtype: int64

<h2> 特征工程 </h2>

下一步是从可能提高性能的现有信息中创建新功能。我还需要执行特征选择，以删除那些跟预测目标无关的特征。

在考虑了安然案例的背景和数据集中包含的信息后，我决定从电子邮件原始数据创建三个新特征。第一个是电子邮件和个人的电子邮件与所有发送给该人的电子邮件的利益相关的比例，第二个是相同的邮件，但是对于poi的邮件的比例，第三个是共享的电子邮件与所有发给该人的电子邮件都有兴趣的人的比率。

这些选择背后的理由是，考虑到个人发送或接收的电子邮件的总数，相对数量可能并不重要。我觉得与poi更多地交流的个人（如电子邮件所示）本身更有可能是poi，因为欺诈行为并不是单独个人形成，应该是一个社交网。当然，也会有部分无辜的人会收到poi的邮件。

In [9]:
# Add the new email features to the dataframe
df['to_poi_ratio'] = df['from_poi_to_this_person'] / df['to_messages']
df['from_poi_ratio'] = df['from_this_person_to_poi'] / df['from_messages']
df['shared_poi_ratio'] = df['shared_receipt_with_poi'] / df['to_messages']

In [10]:
features_list.append('to_poi_ratio')
features_list.append('from_poi_ratio')
features_list.append('shared_poi_ratio')

In [11]:
# Create the new financial features and add to the dataframe
df['bonus_to_salary'] = df['bonus'] / df['salary']
df['bonus_to_total'] = df['bonus'] / df['total_payments'] 

In [12]:
features_list.append('bonus_to_salary')
features_list.append('bonus_to_total') 

<h3> 特征缩放 </h3>

特征缩放可以使机器学习算法工作的更好。比如在K近邻算法中，分类器主要是计算两点之间的距离，如果一个特征比其它的特征有更大的范围值，那么距离将会被这个特征值所主导。因此每个特征应该被归一化，比如将取值范围处理为0到1之间。

In [13]:
from sklearn.preprocessing import scale
# Fill any NaN financial data with a 0
df.fillna(value= 0, inplace=True)

# Create a copy of the dataframe and normalize it to zero mean and unit variance
scaled_df = df.copy()
scaled_df.iloc[:,1:] = scale(scaled_df.iloc[:,1:])


# Send the dataset from dataframe to dictionary for tester.py
my_dataset = scaled_df.to_dict(orient='index')

此时，我还将使用财务数据创建新特征。我认为收到大额奖金的人可能更有可能成为感兴趣的人，因为奖金可能是欺诈活动的结果。将非法资金作为奖金更为轻松，而不是通常涉及合同和股东投入的加薪。这两个新功能将是与薪金相关的奖金，以及与总支付有关的奖金。现在共有19个特征，其中一些功能最有可能是减少或没有任何价值。我将执行功能缩减/选择来优化特征的数量。

In [14]:
# Fill any NaN financial data with a 0
df.fillna(value= 0, inplace=True)

# Create a copy of the dataframe and normalize it to zero mean and unit variance
scaled_df = df.copy()
scaled_df.iloc[:,1:] = scale(scaled_df.iloc[:,1:])

# Send the dataset from dataframe to dictionary for tester.py
my_dataset = scaled_df.to_dict(orient='index')

<h3> 四种算法对比 </h3>

In [40]:
from sklearn.preprocessing import scale
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier
import tester

# Create the classifier, GaussianNB has no parameters to tune
clf = GaussianNB()
tester.dump_classifier_and_data(clf, my_dataset, features_list)
tester.main()

GaussianNB(priors=None)
	Accuracy: 0.82421	Precision: 0.37520	Recall: 0.34650	F1: 0.36028	F2: 0.35188
	Total predictions: 14000	True positives:  693	False positives: 1154	False negatives: 1307	True negatives: 10846



In [16]:
clf = DecisionTreeClassifier()
tester.dump_classifier_and_data(clf, my_dataset, features_list)
tester.main()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.89079	Precision: 0.62641	Recall: 0.58350	F1: 0.60419	F2: 0.59160
	Total predictions: 14000	True positives: 1167	False positives:  696	False negatives:  833	True negatives: 11304



In [17]:
clf = SVC(kernel='linear')
tester.dump_classifier_and_data(clf, my_dataset, features_list)
tester.main()

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
	Accuracy: 0.86043	Precision: 0.51722	Recall: 0.34550	F1: 0.41427	F2: 0.37007
	Total predictions: 14000	True positives:  691	False positives:  645	False negatives: 1309	True negatives: 11355



In [18]:
clf = KMeans(n_clusters=2)
tester.dump_classifier_and_data(clf, my_dataset, features_list)
tester.main()

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)
	Accuracy: 0.77236	Precision: 0.16526	Recall: 0.14650	F1: 0.15531	F2: 0.14990
	Total predictions: 14000	True positives:  293	False positives: 1480	False negatives: 1707	True negatives: 10520



在添加特征之后，所有算法的结果总结如下：

| Classifier            | Precision | Recall  | F1 Score | Accuracy |
|-----------------------|-----------|---------|----------|----------|
| GaussianNB            | 0.37520   | 0.34650 | 0.36028  | 0.82421  |
| DecisionTree          | 0.62641   | 0.58350 | 0.60419  | 0.89079  |
| SVC (kernel='linear') | 0.51722   | 0.34550 | 0.41427  | 0.86043  |
| KMeans (n_clusters=2) | 0.16526   | 0.14650 | 0.15531  | 0.77236  |

决策树的F1最高，其次是支持向量机，在这一点上，k均值不能通过精确度和召回率都满足0.3的需求，因此舍弃该算法。

<h2> 特征选择 </h2>

特征选择是一个重要的数据预处理过程，进行特征选择有两个方面的原因，首先，我们在现实任务重经常会遇到维数灾难问题，这是由于属性过多造成的，其次，去除不相关的特征往往会降低学习任务的难度，留下关键因素。特征选择有三种方法，分别为过滤型，包裹型和内嵌型，我们先看一下决策树和adaboost分类起的功能重要性排序。

In [19]:
clf_tree = DecisionTreeClassifier()
tester.test_classifier(clf_tree, my_dataset, features_list)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
	Accuracy: 0.89036	Precision: 0.62480	Recall: 0.58200	F1: 0.60264	F2: 0.59008
	Total predictions: 14000	True positives: 1164	False positives:  699	False negatives:  836	True negatives: 11301



In [20]:
# Get the feature importances of the DecisionTree Classifier
tree_feature_importances = clf_tree.feature_importances_
tree_features = zip(tree_feature_importances, features_list[1:])
tree_features = sorted(tree_features, key= lambda x:x[0], reverse=True)

# Display the feature names and importance values
print('Tree Feature Importances:\n')
for i in range(10):
    print('{} : {:.4f}'.format(tree_features[i][1], tree_features[i][0]))

Tree Feature Importances:

from_poi_ratio : 0.3897
expenses : 0.2446
shared_receipt_with_poi : 0.1571
to_poi_ratio : 0.0777
shared_poi_ratio : 0.0598
restricted_stock : 0.0359
from_poi_to_this_person : 0.0351
salary : 0.0000
total_payments : 0.0000
bonus : 0.0000


In [21]:
clf_ada = AdaBoostClassifier()
tester.test_classifier(clf_ada, my_dataset, features_list);

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
	Accuracy: 0.91964	Precision: 0.76245	Recall: 0.63550	F1: 0.69321	F2: 0.65739
	Total predictions: 14000	True positives: 1271	False positives:  396	False negatives:  729	True negatives: 11604



In [22]:
# Get the feature importances for the AdaBoost Classifier
ada_feature_importances = clf_ada.feature_importances_
ada_features = zip(ada_feature_importances, features_list[1:])

# Display the feature names and importance values
print('Ada Boost Feature Importances:\n')
ada_features = sorted(ada_features, key=lambda x:x[0], reverse=True)
for i in range(10):
    print('{} : {:.4f}'.format(ada_features[i][1], ada_features[i][0]))

Ada Boost Feature Importances:

from_poi_ratio : 0.1600
shared_receipt_with_poi : 0.1400
to_poi_ratio : 0.1000
shared_poi_ratio : 0.1000
expenses : 0.0800
to_messages : 0.0800
deferred_income : 0.0600
total_stock_value : 0.0600
other : 0.0400
from_this_person_to_poi : 0.0400


从决策树和adaboost的特征重要性上来看，具有不同的结果，接下来我们用SelectKBest并具有自动选择用于分类器的由方差量定义的k最佳特征。

In [23]:
from feature_format import featureFormat, targetFeatureSplit
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

data_dict = featureFormat(my_dataset, features_list)
labels, features = targetFeatureSplit(data_dict)

In [24]:
from sklearn.model_selection import GridSearchCV

n_features = np.arange(1, len(features_list))

# Create a pipeline with feature selection and classification
pipe = Pipeline([
    ('select_features', SelectKBest()),
    ('classify', DecisionTreeClassifier())
])

param_grid = [
    {
        'select_features__k': n_features
    }
]

# Use GridSearchCV to automate the process of finding the optimal number of features
tree_clf= GridSearchCV(pipe, param_grid=param_grid, cv = 10)
tree_clf.fit(features, labels);

In [25]:
tree_clf.best_params_

{'select_features__k': 19}

In [26]:
tree_selection = SelectKBest(k=19)
tree_selection.fit_transform(features, labels)

tree_scores = tree_selection.scores_
tree_features = zip(tree_scores, features_list[1:])
tree_features = sorted(tree_features, key= lambda x:x[0], reverse=True)

print('Tree Feature F-statistics:\n')
for i in range(19):
    print('{} : {:.4f}'.format(tree_features[i][1], tree_features[i][0]))

Tree Feature F-statistics:

bonus : 39.4293
total_stock_value : 26.6422
exercised_stock_options : 26.4094
salary : 26.3390
from_poi_ratio : 25.3142
bonus_to_total : 21.3214
shared_receipt_with_poi : 19.2004
bonus_to_salary : 19.1163
deferred_income : 16.7689
from_poi_to_this_person : 16.3089
shared_poi_ratio : 15.6486
restricted_stock : 10.9824
total_payments : 9.5521
other : 7.7226
expenses : 6.2723
from_this_person_to_poi : 5.4742
to_poi_ratio : 2.7360
to_messages : 2.4327
from_messages : 0.7822


In [27]:
tree_clf = Pipeline([
    ('select_features', SelectKBest(k=19)),
    ('classify', DecisionTreeClassifier()),
])

tester.dump_classifier_and_data(tree_clf, my_dataset, features_list)
tester.main()

Pipeline(steps=[('select_features', SelectKBest(k=19, score_func=<function f_classif at 0x10e858938>)), ('classify', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])
	Accuracy: 0.89329	Precision: 0.63558	Recall: 0.59300	F1: 0.61355	F2: 0.60105
	Total predictions: 14000	True positives: 1186	False positives:  680	False negatives:  814	True negatives: 11320



由于最佳的特征选择k值为19，因此保持所有的特征进行后续分析。

<h2> 选择和调整算法 </h2>

调整第一个算法的参数：

In [28]:
# Create a pipeline with feature selection and classifier
tree_pipe = Pipeline([
    ('select_features', SelectKBest(k=19)),
    ('classify', DecisionTreeClassifier()),
])

# Define the configuration of parameters to test with the 
# Decision Tree Classifier
param_grid = dict(classify__criterion = ['gini', 'entropy'] , 
                  classify__min_samples_split = [2, 4, 6, 8, 10, 20],
                  classify__max_depth = [None, 5, 10, 15, 20],
                  classify__max_features = [None, 'sqrt', 'log2', 'auto'])

# Use GridSearchCV to find the optimal hyperparameters for the classifier
tree_clf = GridSearchCV(tree_pipe, param_grid = param_grid, cv=10)
tree_clf.fit(features, labels)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('select_features', SelectKBest(k=19, score_func=<function f_classif at 0x10e858938>)), ('classify', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'classify__max_features': [None, 'sqrt', 'log2', 'auto'], 'classify__min_samples_split': [2, 4, 6, 8, 10, 20], 'classify__criterion': ['gini', 'entropy'], 'classify__max_depth': [None, 5, 10, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [29]:
# Get the best algorithm hyperparameters for the Decision Tree
tree_clf.best_params_

{'classify__criterion': 'entropy',
 'classify__max_depth': None,
 'classify__max_features': None,
 'classify__min_samples_split': 20}

In [36]:
# Create the classifier with the optimal hyperparameters as found by GridSearchCV
tree_clf = Pipeline([
    ('select_features', SelectKBest(k=19)),
    ('classify', DecisionTreeClassifier(criterion='entropy', max_depth=None, max_features=None, min_samples_split=20))
])

# Test the classifier using tester.py
tester.dump_classifier_and_data(tree_clf, my_dataset, features_list)
tester.main()

Pipeline(steps=[('select_features', SelectKBest(k=19, score_func=<function f_classif at 0x10e858938>)), ('classify', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])
	Accuracy: 0.92036	Precision: 0.72048	Recall: 0.72300	F1: 0.72174	F2: 0.72249
	Total predictions: 14000	True positives: 1446	False positives:  561	False negatives:  554	True negatives: 11439



调整第二个算法的参数：

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
np.seterr(divide='ignore', invalid='ignore')
# Create the pipeline with feature selection and AdaBoostClassifier
ada_pipe = Pipeline([('select_features', SelectKBest(k=19)),
                     ('classify', AdaBoostClassifier())
                    ])

# Define the parameter configurations to test with GridSearchCV
param_grid = dict(classify__base_estimator=[DecisionTreeClassifier(), RandomForestClassifier(), GaussianNB()],
                  classify__n_estimators = [30, 50, 70, 120],
                  classify__learning_rate = [0.5, 1, 1.5, 2, 4])

# Use GridSearchCV to automate the process of finding the optimal parameters
ada_clf = GridSearchCV(ada_pipe, param_grid=param_grid, cv=10)
ada_clf.fit(features, labels)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('select_features', SelectKBest(k=19, score_func=<function f_classif at 0x10e858938>)), ('classify', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'classify__learning_rate': [0.5, 1, 1.5, 2, 4], 'classify__n_estimators': [30, 50, 70, 120], 'classify__base_estimator': [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, ..._score=False, random_state=None,
            verbose=0, warm_start=False), GaussianNB(priors=None)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [33]:
# Display the best parameters for the AdaBoostClassifier
ada_clf.best_params_

{'classify__base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             presort=False, random_state=None, splitter='best'),
 'classify__learning_rate': 4,
 'classify__n_estimators': 70}

In [38]:
# Implement the AdaBoost Classifier with the optimal parameters
ada_clf = Pipeline([('select_features', SelectKBest(k=19)),
                   ('classify', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), learning_rate=4, n_estimators=70))
                   ])

# Test the classifier with cross-validation
tester.dump_classifier_and_data(ada_clf, my_dataset, features_list)
tester.main()

Pipeline(steps=[('select_features', SelectKBest(k=19, score_func=<function f_classif at 0x10e858938>)), ('classify', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,...andom_state=None, splitter='best'),
          learning_rate=4, n_estimators=70, random_state=None))])
	Accuracy: 0.88771	Precision: 0.61432	Recall: 0.57500	F1: 0.59401	F2: 0.58246
	Total predictions: 14000	True positives: 1150	False positives:  722	False negatives:  850	True negatives: 11278



运行算法的最终版本的结果如下所示：


|           算法          | 精确度    | 召回率   | F1 Score | 准确性    |
|------------------------|-----------|--------|----------|----------|
| DecisionTreeClassifier | 0.720     | 0.723  | 0.721    | 0.920    |
| AdaBoostClassifier     | 0.614     | 0.575  | 0.594    | 0.888    |


基于这些结果，我选择的最终算法是DecisionTreeClassifier。我的最终模型由决策树算法，SelectKBest特征选择的19个特征以及上面定义的模型超参数组成。

<h2> 验证和评估 </h2>

最后一步是要对算法进行交叉验证，因为要确认算法是否有效，部分训练集上呈现的有效性，不一定能够在全局有效。在用强大的算法寻找模型的模式时，存在过拟合的风险，也就是训练数据集上得到的结论无法推广到一般情况。因此，评价模型好坏可以用交叉验证的方式。主要可以从以下两个参数度量：

精确度（precision） 顾名思义就是我找的准不准，我找出来的嫌疑人中，有多少是真正的嫌疑人，高的precision，会容易漏掉一些Poi,但不容易误伤好人。

召回率（recall），顾名思义就是反正我的数据查的全不全，放在安然这个案例里，就是我找到的POI全不全。所以，高recall能够做到“宁可误杀好人，也要找到所有嫌疑人”。


In [34]:
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

In [35]:
tester.dump_classifier_and_data(tree_clf, my_dataset, features_list)
tester.main()

Pipeline(steps=[('select_features', SelectKBest(k=19, score_func=<function f_classif at 0x10e858938>)), ('classify', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))])
	Accuracy: 0.90493	Precision: 0.67783	Recall: 0.63750	F1: 0.65705	F2: 0.64518
	Total predictions: 14000	True positives: 1275	False positives:  606	False negatives:  725	True negatives: 11394



我们测试分类器来进行交叉验证，分别从精确度、召回率和F1分数进行了记录。在tester.py中，我看到交叉验证产生k个大小相同的互斥子集，随机种子为了产生可重复的结果，交叉验证参数设置为42。我改变了随机的种子数的时候，模型的表现也降低了。因此，为了防止在交叉验证中随机种子在训练集上做出过度拟合的经典错误，这是需要注意的问题。即使采取预防措施来防止过拟合，我仍然针对一组特定的数据优化了我的模型。为了更好地指出决策树模型的性能指标，我用不同的随机种子进行了10次测试，发现了平均绩效指标。我的模型的最终结果总结如下：

|           算法          | 精确度    | 召回率   | F1 Score | 准确性    |
|------------------------|-----------|--------|----------|----------|
| DecisionTreeClassifier | 0.678     | 0.638  | 0.657    | 0.904    |



<h2> 结论 </h2>

精确度得分为0.678，表明我的模型有67.8%的人被标注为poi，召回率为0.638，表明我的模型里面有63.8%的人是可以确定为poi。

在调整算法的过程中，我认为与其通过不同方式对算法的进行少量调整，也只让F1的分数提高了0.05，不如对提高原始数据的质量。在未来的机器学习模型的开发过程中，我将会集中精力收集尽可能多的高质量数据，再可以考虑训练和调整算法。

### 参考资料

https://github.com/dshgna/ud120-projects/blob/44305deefe627373eb2dcb5e0a1fb22026516458/final_project/poi_id.py

https://github.com/WillKoehrsen/ud120-projects/blob/master/final_project/Enron_Machine_Learning_with_Python.pdf

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

http://blog.csdn.net/xlinsist/article/details/51212348

http://www.cnblogs.com/jasonfreak/p/5448385.html